In [1]:
INPUT_FILEPATH = '/Users/bendoremus/Downloads/2022 Parent Satisfaction Survey (partial results).csv'
DATABASE_SCHEMA = 'sac_survey_2022'

In [2]:
import pandas as pd

from csv import reader as csv_reader
from sqlalchemy import create_engine, text
from pprint import pprint

In [3]:
eng = create_engine('postgresql://bendoremus:@localhost:5432/gvca')
with eng.connect():
    pass

In [4]:
df = pd.read_csv(INPUT_FILEPATH)
df.head(3).T

,0,1,2
Respondent ID,NaN,13345599422.0,13344905413.0
Collector ID,NaN,414834425.0,414834425.0
Start Date,NaN,02/20/2022 07:15:17 PM,02/20/2022 09:05:46 AM
End Date,NaN,02/20/2022 07:24:44 PM,02/20/2022 09:13:19 AM
IP Address,NaN,NaN,NaN
Email Address,NaN,NaN,NaN
First Name,NaN,NaN,NaN
Last Name,NaN,NaN,NaN
Custom Data 1,NaN,NaN,NaN
How many years have you had a child at Golden View Classical Academy? The current academic year counts as 1.,Open-Ended Response,4,6


In [5]:
df['Unnamed: 15'].value_counts()

Extremely Satisfied    40
Satisfied              17
Somewhat Satisfied      8
Not Satisfied           2
Upper School            1
Name: Unnamed: 15, dtype: int64

In [6]:
# get headers, organize columns
with open(INPUT_FILEPATH, 'r') as f_in:
    raw_data_reader = csv_reader(f_in)
    raw_header = raw_data_reader.__next__()
    raw_sub_header = raw_data_reader.__next__()
    ignore_sub_headers = ['Open-Ended Response', 'Response']
    
    # fill empty columns with the appropriate question
    questions = []
    for i, (question, sub_question) in enumerate(zip(raw_header, raw_sub_header)):
        questions += [(i,
                       (question if question else questions[-1][1]),
                       (sub_question if sub_question and sub_question not in ignore_sub_headers else None))]
    
pprint(questions)

[(0, 'Respondent ID', None),
 (1, 'Collector ID', None),
 (2, 'Start Date', None),
 (3, 'End Date', None),
 (4, 'IP Address', None),
 (5, 'Email Address', None),
 (6, 'First Name', None),
 (7, 'Last Name', None),
 (8, 'Custom Data 1', None),
 (9,
  'How many years have you had a child at Golden View Classical Academy?  The '
  'current academic year counts as 1.',
  None),
 (10,
  'Did you or one of your children attend conferences this year?',
  'Grammar School'),
 (11,
  'Did you or one of your children attend conferences this year?',
  'Upper School'),
 (12,
  'Given your children’s education level at the beginning of of the year, how '
  'satisfied are you with their intellectual growth this year?',
  'Grammar School'),
 (13,
  'Given your children’s education level at the beginning of of the year, how '
  'satisfied are you with their intellectual growth this year?',
  'Upper School'),
 (14,
  'How satisfied are you with the education that your children have received '
  'at Golde

In [7]:
# use the output above to relate each column to a question
{
    0: 'respondent_id',
    1: 'collector_id',
    2: 'start_datetime',
    3: 'end_datetime',
    9: (1, 'Tenure at GVCA in years', None),
    10: (2, 'Conference Attendence', 'Grammar'),
    11: (2, 'Conference Attendence', 'Upper'),
    12: (3, 'Intellectual Growth', 'Grammar'),
    13: (3, 'Intellectual Growth', 'Upper'),
    14: (4, 'Education Received', 'Grammar'),
    15: (4, 'Education Received', 'Upper'),
    16: (5, 'Virtues', 'Grammar'),
    17: (5, 'Virtues', 'Upper'),
    18: (6, 'Teacher Communication', 'Grammar'),
    19: (6, 'Teacher Communication', 'Upper'),
    20: (7, 'Leadership Communication', None),
    21: (8, 'Community welcoming', 'Grammar'),
    22: (8, 'Community welcoming', 'Upper'),
    23: (9, "This year's challenges: Teachers", None),
    24: (9, "This year's challenges: Leadership", None),
    25: (9, "This year's challenges: Student Services", None),
    26: (9, "This year's challenges: Child", None),
    27: (10, "Why good choice", "Both"),
    28: (10, "Why good choice", "Grammar"),
    29: (10, "Why good choice", "Upper"),
    30: (11, "Improve", "Both"),
    31: (11, "Improve", "Grammar"),
    32: (11, "Improve", "Upper"),
    33: (12, "Services provided: Economic Assistance", "Grammar"),
    34: (12, "Services provided: Economic Assistance", "Upper"),
    35: (12, "Services provided: IEP", "Grammar"),
    36: (12, "Services provided: IEP", "Upper"),
    37: (12, "Services provided: 504", "Grammar"),
    38: (12, "Services provided: 504", "Upper"),
    39: (12, "Services provided: Gifted", "Grammar"),
    40: (12, "Services provided: Gifted", "Upper"),
    41: (12, "Services provided: Language", "Grammar"),
    42: (12, "Services provided: Language", "Upper"),
    43: (12, "Services provided: Reading", "Grammar"),
    44: (12, "Services provided: Reading", "Upper"),
    45: (12, "Services provided: Math", "Grammar"),
    46: (12, "Services provided: Math", "Upper"),
    47: (12, "Services provided: Behavior", "Grammar"),
    48: (12, "Services provided: Behavior", "Upper"),
    49: (12, "Services provided: Other", "Grammar"),
    50: (12, "Services provided: Other", "Upper"),
    51: (12, "Services provided: Description of Other", "Read text for context"),
    52: (13, "Minority", None),
}
pass

In [18]:
# functions for writing to postgres
def add_to_table(conn, tablename, **kwargs):
    keys = kwargs.keys()
    query = text(
        f'INSERT INTO {tablename} ({", ".join(list(keys))}) '
        f'VALUES ({", ".join([":"+k for k in keys])})'
    )
    conn.execute(query, {**{'tablename': tablename}, **kwargs})
    
    
def split_grammar_upper_rank(conn, question_id, grammar_response, upper_response, **kwargs):
    tablename = 'question_rank'
    if grammar_response:
        # add response
        add_to_table(
            conn=conn, 
            tablename=tablename,
            question_id=question_id,
            grammar=True,
            upper=False,
            response=convert_to_int(grammar_response),
            **kwargs
        )
    if upper_response:
        # add response
        add_to_table(
            conn=conn, 
            tablename=tablename,
            question_id=question_id,
            grammar=False,
            upper=True,
            response=convert_to_int(upper_response),
            **kwargs
        )
        

def open_response_question(conn, sub_questions_and_responses, **kwargs):
    for sub_question_id, response in sub_questions_and_responses.items():
        if response:
            add_to_table(
                conn,
                tablename='question_open_response',
                sub_question_id=sub_question_id,
                response=response,
                **kwargs
            )
            
            
def add_services(conn, grammar_services, upper_services, other_description, **kwargs):
    for k, v in grammar_services.items():
        if v:
            add_to_table(
                conn,
                tablename='question_services_provided',
                grammar=True,
                upper=False,
                service_name=k,
                **kwargs,
            )
    for k, v in upper_services.items():
        if v:
            add_to_table(
                conn,
                tablename='question_services_provided',
                grammar=False,
                upper=True,
                service_name=k,
                **kwargs,
            )
    if other_description:
        add_to_table(
                conn,
                tablename='question_services_provided',
                grammar=False,
                upper=False,
                service_name=f'OTHER DESCRIPTION: {other_description}',
                **kwargs,
            )


def convert_to_bool(value):
    return True if value == 'Yes' else False if value == 'No' else None

def convert_to_int(value):
    if value.startswith('Extremely') or value.startswith('Strongly'):
        return 4
    if value.startswith('Somewhat'):
        return 2
    if value.startswith('Not'):
        return 1
    return 3

In [20]:
with open(INPUT_FILEPATH, 'r') as f_in, eng.connect() as conn:
    raw_data_reader = csv_reader(f_in)
    
    # skip the first two rows
    raw_header = raw_data_reader.__next__()
    raw_sub_header = raw_data_reader.__next__()
    
    conn.execute('BEGIN TRANSACTION;')
    conn.execute(f"SET SCHEMA '{DATABASE_SCHEMA}'")
    
    # TODO: Remove before pushing to prod!
    conn.execute(f'TRUNCATE respondents CASCADE;')
    
    for row in raw_data_reader:
        # respondents, includes demographics and question 1
        add_to_table(
            conn,
            tablename='respondents',
            respondent_id=row[0],
            collector_id=row[1],
            start_datetime=row[2],
            end_datetime=row[3],
            tenure=convert_to_int(row[9]),
            grammar_conferences=convert_to_bool(row[10]),
            upper_conferences=convert_to_bool(row[11]),
            grammar_support=any(row[33:51:2]),
            upper_support=any(row[34:51:2]),
            any_support=any(row[34:52]),
            minority=convert_to_bool(row[52]),
        )
        
        # question 3:
        # Given your children’s education level at the beginning of of the year, how satisfied are you with their intellectual growth this year?
        split_grammar_upper_rank(
            conn,
            question_id='3',
            respondent_id=row[0],
            grammar_response=row[12],
            upper_response=row[13],
        )
        
        # question 4:
        # How satisfied are you with the education that your children have received at Golden View Classical Academy this year?	
        split_grammar_upper_rank(
            conn,
            question_id='4',
            respondent_id=row[0],
            grammar_response=row[14],
            upper_response=row[15],
        )
        
        # question 5:
        # GVCA emphasizes 7 core virtues: Courage, Moderation, Justice, Responsibility, Prudence, Friendship, and Wonder. How strongly is the school culture influenced by those virtues?
        split_grammar_upper_rank(
            conn,
            question_id='5',
            respondent_id=row[0],
            grammar_response=row[16],
            upper_response=row[17],
        )
        
        # question 6:
        # How effective is the communication between your family and your childrens' teachers?
        split_grammar_upper_rank(
            conn,
            question_id='6',
            respondent_id=row[0],
            grammar_response=row[18],
            upper_response=row[19],
        )
        
        # question 7:
        # How effective is the communication between your family and the school leadership?	
        if row[20]:
            add_to_table(
                conn,
                tablename='question_rank',
                respondent_id=row[0],
                question_id='7',
                grammar=False,
                upper=False,
                response=convert_to_int(row[20]),
            )
        
        # question 8:
        # How welcoming is the school community?
        split_grammar_upper_rank(
            conn,
            question_id='8',
            respondent_id=row[0],
            grammar_response=row[21],
            upper_response=row[22],
        )
        
        # question 9:
        # Given this year's challenges, what are your thoughts on the following aspects of the school environment?
        open_response_question(
            conn,
            sub_questions_and_responses={
                'teacher': row[23],
                'leadership': row[24],
                'student_services': row[25],
                'child': row[26],
            },
            respondent_id=row[0],
            question_id='9',
        )
        
        # question 10:
        # What makes GVCA a good choice for you and your family?
        open_response_question(
            conn,
            sub_questions_and_responses={
                'both': row[27],
                'grammar': row[28],
                'upper': row[29],
            },
            respondent_id=row[0],
            question_id='10',       
        )
        
        # question 11:
        # Please provide us with examples of how GVCA can better serve you and your family.
        open_response_question(
            conn,
            sub_questions_and_responses={
                'both': row[30],
                'grammar': row[31],
                'upper': row[32],
            },
            respondent_id=row[0],
            question_id='11',
        )
        
        # question 12:
        # What services have your children received at Golden View this school year? Please check all that apply.
        add_services(
            conn,
            respondent_id=row[0],
            question_id='12',
            grammar_services={
                'Qualify for Economic Assistance': row[33],
                'IEP and related services, including Resource Class, Psychological services, Speech/Lanugage services, and Occupational therapy services': row[35],
                'Section 504 Plan and related services': row[37],
                'Gifted and Talented Programming/Advanced Learning Plans': row[39],
                'English Language Learning Services': row[41],
                'Reading Intervention': row[43],
                'Math Intervention': row[45],
                'Student Behavior/Counseling-type services': row[47],
                'Other': row[49],
            },
            upper_services={
                'Qualify for Economic Assistance': row[34],
                'IEP and related services, including Resource Class, Psychological services, Speech/Lanugage services, and Occupational therapy services': row[36],
                'Section 504 Plan and related services': row[38],
                'Gifted and Talented Programming/Advanced Learning Plans': row[40],
                'English Language Learning Services': row[41],
                'Reading Intervention': row[44],
                'Math Intervention': row[46],
                'Student Behavior/Counseling-type services': row[48],
                'Other': row[50],
            },
            other_description=row[51],
        )

    conn.execute('END TRANSACTION;')